In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

In [2]:
@dataclass
class Message:
    content: str

In [4]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

In [5]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [6]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [7]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

In [8]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")

In [9]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [10]:
display(Markdown(response.content))

## Pros of AutoGen:
AutoGen offers several advantages for AI Agent projects:

1. **Scalability**: Its modular and extensible framework allows for the creation of scalable and customizable systems, accommodating the growth of projects over time.

2. **Ease of Use**: Integrated observability and debugging tools simplify the monitoring and control of agent workflows, making it easier for developers to manage complex interactions between agents.

3. **Multi-Agent Collaboration**: AutoGen excels in creating teams of AI agents that can collaborate either autonomously or with human input, enhancing the capabilities of your AI system.

4. **Natural Language Handoffs**: The framework reduces coordination complexity by allowing agents to transfer information seamlessly without requiring custom inter-agent protocols, simplifying communication.

5. **Backed by Microsoft**: With support from Microsoft, AutoGen benefits from substantial development resources and community usage scenarios, ensuring robust use cases and support.

6. **Flexible and Conversation-Driven**: It enables dynamic, automated chat-based interactions among agents, allowing for sophisticated task-oriented collaborations.

These pros make AutoGen a compelling choice for developing complex AI systems that require coordination and collaboration among multiple agents. 

TERMINATE

## Cons of AutoGen:
Here are several reasons against choosing AutoGen for your AI Agent project:

1. **Steep Learning Curve**: Users have reported that mastering AutoGen requires significant time and effort due to its complex features and graph-based approach.

2. **Limited Documentation**: The documentation is often considered hard to read, lacking sufficient examples, which can hinder effective learning and implementation.

3. **Deployment Challenges**: AutoGen may not be easy to deploy without substantial customization, making it less user-friendly, especially for teams lacking technical expertise.

4. **Inconsistent Functionality**: Certain features, like structured outputs, reportedly do not work as intended, which could impact the reliability of your project.

5. **Lack of User Interface Support**: As a CLI-first tool with limited user interface options, it may not be suitable for projects requiring a more accessible or visual interaction layer.

These factors could influence the decision to use AutoGen, especially if ease of use and reliable functionality are critical for your project. 

TERMINATE



## Decision:

Based on the research from the team, I recommend using AutoGen for the project. The advantages of scalability, ease of use in monitoring, and robust multi-agent collaboration significantly outweigh the drawbacks. While there are challenges like the steep learning curve and limited documentation, the pros highlight AutoGen's ability to handle complex interactions and its backing by Microsoft, which provides assurance of support and development. The potential for sophisticated task-oriented collaborations makes it an appealing choice for projects that prioritize innovation and growth.

TERMINATE

In [11]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [12]:
await host.stop()